In [22]:
from PIL import Image, ImageDraw, ImageFont
import sys
import numpy as np

CHARACTER_WIDTH=14
CHARACTER_HEIGHT=25


def load_letters(fname):
    im = Image.open(fname)
    px = im.load()
    (x_size, y_size) = im.size
    print(im.size)
    print(int(x_size / CHARACTER_WIDTH) * CHARACTER_WIDTH)
    result = []
    for x_beg in range(0, int(x_size / CHARACTER_WIDTH) * CHARACTER_WIDTH, CHARACTER_WIDTH):
        result += [ [ "".join([ '*' if px[x, y] < 1 else ' ' for x in range(x_beg, x_beg+CHARACTER_WIDTH) ]) for y in range(0, CHARACTER_HEIGHT) ], ]
    return result

def load_training_letters(fname):
    TRAIN_LETTERS="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789(),.-!?\"' "
    letter_images = load_letters(fname)
    return { TRAIN_LETTERS[i]: letter_images[i] for i in range(0, len(TRAIN_LETTERS) ) }

def get_training_arrays(letters):
    #initialize array
    arrays = {}
    #for each letter in the train set
    for letter in letters.keys():
        #take each line of pixels, transla and add them to a 1d numpy array
        arr = np.array([])
        for line in letters[letter]:
            arr = np.append(arr,np.array([1 if p == "*" else 0 for p in line]), axis = 0)
        arrays[letter] = arr
    
    return arrays

def get_test_array(letter):
    #same process, add each line of pixels as a 1 or 0
    array = np.array([])
    for line in letter:
        array = np.append(array, np.array([1 if p == "*" else 0 for p in line]), axis = 0)
    
    return array

def get_class(train, test):
    train_letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789(),.-!?\"' "
    probs = np.array([])
    for index, letter in enumerate(train.keys()):
        matches = (test == train[letter]).sum()
        mismatches = (CHARACTER_HEIGHT * CHARACTER_WIDTH) - (test == train[letter]).sum()
        probs = np.append(probs, .99 * (matches / (CHARACTER_HEIGHT * CHARACTER_WIDTH)) + 0)
        
    best_match_ix = np.argmax(probs)
    
    return train_letters[best_match_ix]

def simple_model(train_set, test_set):
    prediction = ""
    train_arrays = get_training_arrays(train_set)
    for l in test_set:
        test_array = get_test_array(l)
        best_match = get_class(train_arrays, test_array)[0]
        prediction += best_match
    
    return prediction

def read_data(fname):
    exemplars = []
    file = open(fname, 'r');
    for line in file:
        data = [w for w in line.split()[0::2]]
        exemplars += data

    return exemplars

def read_data2(fname):
    exemplars = []
    file = open(fname, 'r');
    for line in file:
        data = [w for w in line.split()]
        exemplars += data

    return exemplars

def get_probs(txt):
    train_letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789(),.-!?\"' "
    sentences = []
    sentence = ""
    for word in txt:
        if word == ".":
            sentence = sentence.rstrip()
            sentence += word
            sentences.append(sentence)
            sentence = ""
        elif word == ',':
            sentence = sentence.rstrip()
            sentence += word + " "
        elif word == '``':
            sentence += word
        elif word == "''":
            sentence = sentence.rstrip()
            sentence += word + " "
        elif word in ['The', 'the', "It", 'it']:
            continue
        else:
            sentence += word + " "
    lstr = " ".join(sentences)
    
    #print(lstr[0:30])
    
    tr = {}
    letter_counts = {}
    for char in train_letters:
        transitions = []
        #for sentence in sentences:
        for index, letter in enumerate(lstr):
            if letter not in letter_counts.keys():
                letter_counts[letter] = 1
            letter_counts[letter] += 1
            if index + 1 == len(lstr):
                break
            if letter == char:
                transitions.append(lstr[index+1])
        
        trprobs = {}
        for letter in transitions:
            if letter not in trprobs.keys():
                trprobs[letter] = transitions.count(letter)/len(transitions)#/letter_counts[letter]
        tr[char] = trprobs
    
    init = {}
    initials = []
    for sentence in sentences:
        initials.append(sentence[0])
        
    for letter in initials:
        if letter not in init.keys():
            init[letter] = initials.count(letter) / len(initials)
    
    return tr, init, letter_counts

def get_test_arrays(letters):
    arrays = {}
    for letter in range(len(letters)):
        arr = np.array([])
        for line in letters[letter]:
            arr = np.append(arr,np.array([1 if p == "*" else 0 for p in line]), axis = 0)
        arrays[letter] = arr
    
    return arrays

def viterbi(train_arrays, test_arrays, tr, init):
    train_letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789(),.-!?\"' "
    prediction = ""
    t_min1= ()

    for l in test_arrays:
        if l == 0:
            probs = np.array([])
            for letter in train_letters:
                pct_match = (test_arrays[0] == train_arrays[letter]).sum() / (CHARACTER_HEIGHT * CHARACTER_WIDTH)
                matches = (test_arrays[0] == train_arrays[letter]).sum()
                probs = np.append(probs, (.99 * pct_match*2) * init[letter] if letter in init.keys() else 1e-50)
                #probs = np.append(probs, matches * init[letter] if letter in init.keys() else 1e-50)
                print("Letter: ", letter, "| Emission: ", (.98 * pct_match), "| Initial Prob: ", init[letter] if letter in init.keys() else 1e-100)
            
            pred_letter = train_letters[np.argmax(probs)]
            prediction += pred_letter
            t_min1 = (pred_letter, np.max(probs))
            print(t_min1)
        else:    
            probs = np.array([])
            for letter in train_letters:
                pct_match = (test_arrays[l] == train_arrays[letter]).sum() / (CHARACTER_HEIGHT * CHARACTER_WIDTH)
                matches = (test_arrays[l] == train_arrays[letter]).sum()
                probs = np.append(probs, t_min1[1] * (.99 * pct_match*2) * tr[t_min1[0]][letter] if letter in tr[t_min1[0]].keys() else 1e-50)
                
            pred_letter = train_letters[np.argmax(probs)]
            prediction += pred_letter
            t_min1 = (pred_letter, np.max(probs))
            print(t_min1)
    return prediction
    

In [17]:
(0.8+.25) * .1-.25

-0.145

In [18]:
(0.7+ .25) * .2-.25

-0.06

In [20]:
txt = read_data('bc.train')
train = load_training_letters('courier-train.png')
test = load_letters('test-0-0.png')
#test = load_letters('test-17-0.png')

(1008, 25)
1008
(477, 25)
476


In [21]:
simple_model(train, test)

'SUPREME COURT OF THF UN1TED STATES'

In [23]:
train_arrays = get_training_arrays(train)
test_arrays = get_test_arrays(test)
txt = read_data('12345.txt')
tr, init, letter_counts = get_probs(txt)

In [27]:
tr['T']

{'h': 0.6143170197224251,
 'a': 0.041344046749452155,
 'u': 0.025712198685171658,
 'e': 0.05712198685171658,
 'o': 0.10372534696859022,
 'y': 0.002337472607742878,
 'E': 0.00014609203798392987,
 'w': 0.019138056975894813,
 'r': 0.06267348429510591,
 'O': 0.005113221329437546,
 '.': 0.007450693937180424,
 'i': 0.028487947406866325,
 'C': 0.0020452885317750183,
 'N': 0.0005843681519357195,
 ' ': 0.007304601899196494,
 's': 0.0027757487216946678,
 'V': 0.008181154127100073,
 'A': 0.002191380569758948,
 ',': 0.0014609203798392988,
 'U': 0.001168736303871439,
 'c': 0.0004382761139517896,
 "'": 0.00029218407596785974,
 'R': 0.0010226442658875091,
 'H': 0.0004382761139517896,
 '-': 0.00029218407596785974,
 'S': 0.0030679327976625274,
 '}': 0.00014609203798392987,
 'F': 0.0005843681519357195,
 'P': 0.00029218407596785974,
 ')': 0.00014609203798392987}

In [ ]:
init

In [26]:
viterbi(train_arrays, test_arrays, tr, init)

Letter:  A | Emission:  0.756 | Initial Prob:  0.09536541889483066
Letter:  B | Emission:  0.8231999999999999 | Initial Prob:  0.046686589074132326
Letter:  C | Emission:  0.8484 | Initial Prob:  0.020263185488098984
Letter:  D | Emission:  0.84 | Initial Prob:  0.01774667086085771
Letter:  E | Emission:  0.8092 | Initial Prob:  0.017196183286148685
Letter:  F | Emission:  0.812 | Initial Prob:  0.031587501310684704
Letter:  G | Emission:  0.8372 | Initial Prob:  0.008807801195344448
Letter:  H | Emission:  0.8372 | Initial Prob:  0.0731624200482332
Letter:  I | Emission:  0.8064 | Initial Prob:  0.08472265911712279
Letter:  J | Emission:  0.8204 | Initial Prob:  0.0053475935828877
Letter:  K | Emission:  0.7784 | Initial Prob:  0.002962147425815246
Letter:  L | Emission:  0.7979999999999999 | Initial Prob:  0.013159274404949146
Letter:  M | Emission:  0.7448 | Initial Prob:  0.033396246199014365
Letter:  N | Emission:  0.7979999999999999 | Initial Prob:  0.02191464821222607
Letter:  O

'The an an an an an an an an an an '

In [ ]:
#!/usr/bin/python
#
# Perform optical character recognition, usage:
#     python3 ./image2text.py train-image-file.png train-text.txt test-image-file.png
# 
# Authors: (insert names here)
# (based on skeleton code by D. Crandall, Oct 2020)
#

from PIL import Image, ImageDraw, ImageFont
import sys

CHARACTER_WIDTH=14
CHARACTER_HEIGHT=25


def load_letters(fname):
    im = Image.open(fname)
    px = im.load()
    (x_size, y_size) = im.size
    print(im.size)
    print(int(x_size / CHARACTER_WIDTH) * CHARACTER_WIDTH)
    result = []
    for x_beg in range(0, int(x_size / CHARACTER_WIDTH) * CHARACTER_WIDTH, CHARACTER_WIDTH):
        result += [ [ "".join([ '*' if px[x, y] < 1 else ' ' for x in range(x_beg, x_beg+CHARACTER_WIDTH) ]) for y in range(0, CHARACTER_HEIGHT) ], ]
    return result

def load_training_letters(fname):
    TRAIN_LETTERS="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789(),.-!?\"' "
    letter_images = load_letters(fname)
    return { TRAIN_LETTERS[i]: letter_images[i] for i in range(0, len(TRAIN_LETTERS) ) }

#####
# main program
if len(sys.argv) != 4:
    raise Exception("Usage: python3 ./image2text.py train-image-file.png train-text.txt test-image-file.png")

(train_img_fname, train_txt_fname, test_img_fname) = sys.argv[1:]
train_letters = load_training_letters(train_img_fname)
test_letters = load_letters(test_img_fname)

## Below is just some sample code to show you how the functions above work. 
# You can delete this and put your own code here!


# Each training letter is now stored as a list of characters, where black
#  dots are represented by *'s and white dots are spaces. For example,
#  here's what "a" looks like:
print("\n".join([ r for r in train_letters['a'] ]))

# Same with test letters. Here's what the third letter of the test data
#  looks like:
print("\n".join([ r for r in test_letters[2] ]))



# The final two lines of your output should look something like this:
print("Simple: " + "Sample s1mple resu1t")
print("   HMM: " + "Sample simple result") 